# **Deforum Google Colab**
Deforum for Google Colab. Generate animations with diffusion models. Features include: text-to-video, image-to-video, video-to-video, and keyframe camera motions. Supports SD1.5, SDXL, and SD3. Built on top of diffusers. MIT license. Notebook by [Huemin](https://www.huemin.art)

In [1]:
#@markdown **SETUP ENVIRONMENT**
import subprocess, os, sys
sub_p_res = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total,memory.free', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(f"{sub_p_res[:-1]}")

models_path = "/content/models"
output_path = "/content/output"

# Mount Google Drive
mount_google_drive = True
force_remount = False

if mount_google_drive:
    from google.colab import drive
    try:
        drive_path = "/content/drive"
        drive.mount(drive_path,force_remount=force_remount)
        models_path_gdrive = "/content/drive/MyDrive/AI/models"
        output_path_gdrive = "/content/drive/MyDrive/AI/outputs"
        models_path = models_path_gdrive
        output_path = output_path_gdrive
    except:
        print("...error mounting drive or with drive path variables")
        print("...reverting to default path variables")

import os
os.makedirs(models_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)

print(f"models_path: {models_path}")
print(f"output_path: {output_path}")

# install stuff
!git clone https://github.com/huemin-art/deforum-google-colab.git
!pip install -q diffusers transformers accelerate

FileNotFoundError: [Errno 2] No such file or directory: 'nvidia-smi'

In [5]:
import torch
from pydantic import BaseModel
from diffusers import AutoPipelineForText2Image, AutoPipelineForImage2Image

sys.path.append(os.path.join(os.getcwd(), "deforum-google-colab"))

from helpers import load_image, create_animation_from_pil_images

#@markdown **MODEL SETTINGS**
class DeforumConfig(BaseModel):
    model: str = "stabilityai/stable-diffusion-xl-base-1.0" #@param {type:"string"}
    device: str = "cuda"
    variant: str = "fp16"
    safetensors: bool = True
    torch_dtype: str = "float16"

#@markdown **IMAGE SETTINGS**
class DeforumArgs(BaseModel):
    prompt: str = "cat sushi" #@param {type:"string"}
    negative_prompt: str = "" #@param {type:"string"}
    width: int = 512 #@param {type:"integer"}
    height: int = 512 #@param {type:"integer"}
    num_inference_steps: int = 20 #@param {type:"integer"}
    guidance_scale: float = 4.0 #@param {type:"number"}
    strength: float = 0.5 #@param {type:"number"}
    init_image: str = "" #@param {type:"string"}

class DeforumAnimationArgs(BaseModel):
#@markdown **ANIMATION SETTINGS**
    duration_seconds: int = 30 #@param {type:"integer"}
    frames_per_second: int = 12 #@param {type:"integer"}
    max_frames = duration_seconds*frames_per_second

class DeforumGenerator:
    def __init__(self, config: DeforumConfig):
        self.config = config
        self.txt2img = AutoPipelineForText2Image.from_pretrained(
            self.config.model,
            torch_dtype=getattr(torch, self.config.torch_dtype),
            variant=self.config.variant,
            use_safetensors=self.config.safetensors
        ).to(self.config.device)
        self.img2img = AutoPipelineForImage2Image.from_pipe(self.txt2img)

        # state
        self.init = None
        self.frames = []

    def generate_image(self, args: DeforumArgs):

        if self.init is None
            self.init = load_image(args.init_image)

        if self.init is None:
            image = self.txt2img(
                prompt=args.prompt,
                negative_prompt=args.negative_prompt,
                width=args.width,
                height=args.height,
                num_inference_steps=args.num_inference_steps,
                guidance_scale=args.guidance_scale
            ).images[0]
        else:
            image = self.img2img(
                prompt=args.prompt,
                negative_prompt=args.negative_prompt,
                width=args.width,
                height=args.height,
                num_inference_steps=args.num_inference_steps,
                guidance_scale=args.guidance_scale,
                strength=args.strength
            ).images[0]
        return image

    def generate_animation(self, args: DeforumAnimationArgs):

        # reset
        self.init = None
        self.frames = []

        # loop
        for i in range(args.max_frames):
            self.init = self.generate_image(args)
            self.frames.append(self.init)

        # video
        create_animation_from_pil_images(
            self.frames,
            "test.mp4",
            fps=args.frames_per_second,
            loop=0,
            crf=17
        )

if __name__ == "__main__":
    if "gen" not in globals():
        config = DeforumConfig()
        gen = DeforumGenerator(config)

    args = DeforumArgs()
    img = gen(args)
    img.save("output.png")

  0%|          | 0/20 [00:00<?, ?it/s]